<a href="https://colab.research.google.com/github/Xornotor/Trabalho_IA_2022-2/blob/main/AndrePaiva_CarlosCerqueira_Trabalho_IA_2022_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Final: MATA64 - Inteligência Artificial - UFBA 2022.2**
# **Alunos:** André Paiva e Carlos Cerqueira

## 1 - Instalação e configuração de dependências

Utilizamos a biblioteca **Manim** para facilitar a visualização dos algoritmos desenvolvidos. O seguinte trecho de código deve ser executado e após sua execução o Runtime deve ser reinicializado.

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Abaixo, são importadas as bibliotecas utilizadas neste Notebook.

In [1]:
import numpy as np
import pandas as pd
from manim import *
from random import random

Manim Community v0.17.1

## 2 - Algoritmo MiniMax para resolução de Jogo da velha

`makeBoard` é uma função auxiliar feita apenas para imprimir o "tabuleiro".

In [2]:
def makeBoard(board):
    for i in range (0,9):
        if((i>0) and (i%3)==0):
            print();
        if(board[i]==0):
            print("- ", end="")
        if (board[i]==1):
            print("O ", end="")
        if(board[i]==-1):    
            print("X ", end="")
    print()

Define as funções para pegar e validar o input do usuário

In [3]:
def UserTurn(board):
    pos=int(input("Valor de X em [1...9]: "))

    # Confere caso a casa esteja ocupada
    if not (board[pos-1]==0):
        print("Movimento invalido")
        return False
    board[pos-1]=-1
    return True

Retorna 0 caso o jogo continue ou indica o vencedor do jogo caso exista

In [4]:
def analyzeBoard(board):
    victoryPositions=[
        [0,1,2],
        [3,4,5],
        [6,7,8],
        [0,3,6],
        [1,4,7],
        [2,5,8],
        [0,4,8],
        [2,4,6]
      ]

    for i in range(0,8):
        # Conefere se uma das posições de vitoria ocorreu
        if(board[victoryPositions[i][0]] != 0 and
           board[victoryPositions[i][0]] == board[victoryPositions[i][1]] and
           board[victoryPositions[i][0]] == board[victoryPositions[i][2]]):
            return board[victoryPositions[i][2]]
    return 0

A Função `MinMax` vai calcular, com base em uma pontuação, qual é a melhor posição para o computador jogar.

Para isso, é bom prestar atenção em duas variaveis especificas, `pos`, que se refere a posição que sera jogado, e `value` que é a pontuação associada com essa posição. Assim, o algoritmo vai escolher a posição que tenha um maior valor associado.

In [5]:
def minMax(board,player):
    # Check if the player will win the game
    x=analyzeBoard(board)
    if(x!=0):
        return (x*player)

    pos=-1
    value=-2

    for i in range(0,9):
        if(board[i]==0):
            board[i]=player
            score=-minMax(board,(player*-1))
            if(score>value):
                value=score
                pos=i
            board[i]=0

    if(pos==-1):
        return 0
    return value

In [6]:
def ComputerTurn(board):
    animationArr = []
    pos=-1
    value=-1
    for i in range(0,9):
        if(board[i]==0):
            board[i]=1
            score=-minMax(board, -1)
            animationArr.append(score)
            board[i]=0
            if(score>value):
                value=score
                pos=i
 
    board[pos]=1
    return animationArr

In [7]:
board=[0,0,0,0,0,0,0,0,0]
animationArr = []
print("MinMax : O Vs. Jogador : X")
for i in range (0,9):
  if(analyzeBoard(board)!=0):
    break
  if((i+1)%2==0):
    animationPosArr = ComputerTurn(board)
    animationArr.append(animationPosArr)
  else:
    makeBoard(board)
    userValidInput = UserTurn(board)
    if not userValidInput:
      print("Jogo encerrado!")
      break

x=analyzeBoard(board)
if(x==0 and userValidInput):
  makeBoard(board)
  print("Empate")
if(x==-1 and userValidInput):
  makeBoard(board)
  print("Jogador ganha")
if(x==1 and userValidInput):
  makeBoard(board)
  print("MinMax ganha")

MinMax : O Vs. Jogador : X
- - - 
- - - 
- - - 
Valor de X em [1...9]: 3
- - X 
- O - 
- - - 
Valor de X em [1...9]: 1
X O X 
- O - 
- - - 
Valor de X em [1...9]: 8
X O X 
O O - 
- X - 
Valor de X em [1...9]: 6
X O X 
O O X 
- X O 
Valor de X em [1...9]: 7
X O X 
O O X 
X X O 
Empate


Podemos ver a animação do algoritmo da seguinte maneira:

In [9]:
%%manim -v WARNING --disable_caching -qm ManimMinmax

import networkx as nx

class ManimMinmax(Scene):
    def construct(self):
        tree_layout_scale = 3
        circle = Circle(radius=0.3,color=RED).move_to([0,1.25, 0])
        end_point = (-2.35,-1.25,0)
        animation = ApplyMethod(circle.shift, end_point)

        G = nx.Graph()
        G.add_node(0)

        nodes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        edges = [(0, 1), (0, 2),(0, 3), (1, 4), (2, 5), (3, 6), (1, 7), (2, 8), (3, 9)]


        for node in nodes:
            G.add_node(node)
        for edge in edges:
            G.add_edge(edge[0], edge[1])

        # Search trees
        
        pos_dot = LabeledDot(label=Text("+1", font_size=16, color=BLACK), radius=0.2)
        neg_dot = LabeledDot(label=Text("-1", font_size=16, color=BLACK), radius=0.2)
        zero_dot = LabeledDot(label=Text("0", font_size=16, color=BLACK), radius=0.2)
        user_dot = LabeledDot(label=Text("U", font_size=16, color=BLACK), radius=0.2)
        
        title = Text("Minimax").scale(0.6).shift(3 * UP)
        
        tree = Graph(list(G.nodes), 
                  list(G.edges), 
                  layout="tree", 
                  root_vertex=0,
                  labels=False,
                  layout_scale=tree_layout_scale,
                               vertex_mobjects = {
                                   0: user_dot.copy(),
                                   1: zero_dot.copy(),
                                   2: neg_dot.copy(),
                                   3: zero_dot.copy(),
                                   4: user_dot.copy(),
                                   5: user_dot.copy(),
                                   6: user_dot.copy(),
                                   7: user_dot.copy(),
                                   8: user_dot.copy(),
                                   9: user_dot.copy()
                               })
        
        self.add(tree, title)
        self.play(animation)
        self.wait(2)

## 3 - Perceptron aplicado à classificação de imóveis para recomendação

Para este problema, consideremos o seguinte cenário:



> Um cliente em um aplicativo de venda/aluguel de imóveis avaliou alguns dos imóveis anunciados, num sistema similar a "like/dislike", no qual o cliente deixa registrado se gostou ou não gostou. A partir destas avaliações, queremos implementar um perceptron para prever se o cliente gostaria ou não de outros imóveis que ele não avaliou.



Criamos um modelo de **perceptron** utilizando um dicionário, cujas chaves são *W* e *b*, sendo que:


*   `perceptron['W']` retorna um `np.array` contendo os pesos;
*   `perceptron['b']` retorna o bias.


Foram definidas as funções `create_perceptron`, `predict` e `train_step`, cujas respectivas funções são: 


*   Criar um perceptron e inicializar seus pesos e bias;
*   Efetuar a predição do valor esperado com base em features de entrada;
*   Efetuar o treinamento dos pesos e bias do perceptron.




In [10]:
# Função de inicialização do perceptron com pesos e bias aleatórios
def create_perceptron(features):
  perceptron = {}
  perceptron['W'] = np.array([random() for i in range(features)])
  perceptron['b'] = random()
  return perceptron

# Função de predição do perceptron
def predict(input, perceptron, threshold = 0):
  linear_prediction = np.dot(perceptron['W'], input) + perceptron['b']
  return int(linear_prediction >= threshold)

# Função de treinamento do perceptron
def train_step(input, output, perceptron, learning_rate=1e-1):
  param_change = False
  pred = predict(input, perceptron)
  if(pred != int(output)):
    param_change = True
    perceptron['W'] = perceptron['W'] + (learning_rate * (output - pred) * input)
    perceptron['b'] = perceptron['b'] + (learning_rate * (output - pred))
  return perceptron, param_change

A seguir, temos dois dataframes com dados de algumas casas à venda, sendo um dataframe de **treino** e um dataframe de **testes**.

O data frame de **treino** contém as seguintes informações:
*   Área da casa (m²);
*   Idade da construção (anos);
*   Número de Quartos;
*   Número de Banheiros;
*   Avaliação do cliente (1 = Gostou, 0 = Não gostou).

O data frame de **testes** contém:
*   Área da casa (m²);
*   Idade da construção (anos);
*   Número de Quartos;
*   Número de Banheiros.



In [11]:
#Dataframe de treinamento
train_df = pd.DataFrame(np.array([
                            [287.00, 20, 3, 3, True],
                            [57.00, 30, 2, 1, False],
                            [89.00, 22, 3, 2, False],
                            [210.00, 60, 5, 3, True],
                            [157.00, 40, 4, 3, False],
                            [250.00, 28, 4, 2, True]
                          ]),                        
                        columns=['Área','Idade','Quartos','Banheiros','Avaliação']
                        )

#Dataframe de teste
test_df = pd.DataFrame(np.array([
                            [242.00, 23, 4, 2],
                            [197.00, 15, 3, 3],
                            [67.00, 32, 2, 1],
                            [94.00, 27, 3, 3],
                            [138.00, 5, 2, 3],
                            [314.00, 50, 4, 5]
                          ]),                        
                        columns=['Área','Idade','Quartos','Banheiros']
                        )

In [12]:
#Exibição do dataframe de treino
train_df

,Área,Idade,Quartos,Banheiros,Avaliação
0,287.0,20.0,3.0,3.0,1.0
1,57.0,30.0,2.0,1.0,0.0
2,89.0,22.0,3.0,2.0,0.0
3,210.0,60.0,5.0,3.0,1.0
4,157.0,40.0,4.0,3.0,0.0
5,250.0,28.0,4.0,2.0,1.0


In [13]:
#Exibição do dataframe de testes
test_df

,Área,Idade,Quartos,Banheiros
0,242.0,23.0,4.0,2.0
1,197.0,15.0,3.0,3.0
2,67.0,32.0,2.0,1.0
3,94.0,27.0,3.0,3.0
4,138.0,5.0,2.0,3.0
5,314.0,50.0,4.0,5.0


A seguir, é feita a criação e inicialização de um perceptron e a rotina de treinamento.

In [14]:
#Valores úteis
examples = train_df.shape[0]
features = train_df.shape[1] - 1

#Data stream (para visualização)
data_stream = {'input': [],
               'perceptron': [],
               'output': []}

#Inicialização do perceptron
perceptron = create_perceptron(features)

#Treinamento
param_stability = 0
while(True):
  for j in range(examples):
    input = train_df.iloc[j, 0:features].to_numpy()
    output = train_df.iloc[j, train_df.shape[1] - 1]
    data_stream['input'].append(input.tolist())
    data_stream['output'].append(output)
    data_stream['perceptron'].append(perceptron.copy())
    perceptron, param_change = train_step(input, output, perceptron)
    if(param_change == False):
      param_stability += 1
    else:
      param_stability = 0
  if(param_stability >= examples):
    data_stream['input'].append(input.tolist())
    data_stream['output'].append(output)
    data_stream['perceptron'].append(perceptron.copy())
    break

Após o treinamento, mostramos novamente o dataset de treino, agora com adição da coluna de preços preditos pelo perceptron. Podemos notar uma discrepância significativa, apesar de uma relativa proximidade entre os valores real e predito.

In [15]:
#Predições do dataframe de treinamento
train_predictions = np.zeros((train_df.shape[0]))

for j in range(train_df.shape[0]):
    input = train_df.iloc[j, 0:features].to_numpy()
    train_predictions[j] = predict(input, perceptron)

train_df_with_predictions = train_df.copy()
train_df_with_predictions['Avaliação predita'] = train_predictions

train_df_with_predictions

,Área,Idade,Quartos,Banheiros,Avaliação,Avaliação predita
0,287.0,20.0,3.0,3.0,1.0,1.0
1,57.0,30.0,2.0,1.0,0.0,0.0
2,89.0,22.0,3.0,2.0,0.0,0.0
3,210.0,60.0,5.0,3.0,1.0,1.0
4,157.0,40.0,4.0,3.0,0.0,0.0
5,250.0,28.0,4.0,2.0,1.0,1.0


Agora, mostramos novamente o dataset de testes, mas agora com as predições feitas pelo perceptron após o treinamento.

In [16]:
#Predições do dataframe de testes
test_predictions = np.zeros((test_df.shape[0]))

for j in range(test_df.shape[0]):
    input = test_df.iloc[j].to_numpy()
    test_predictions[j] = predict(input, perceptron)

test_df_with_predictions = test_df.copy()
test_df_with_predictions['Avaliação predita'] = test_predictions

test_df_with_predictions

,Área,Idade,Quartos,Banheiros,Avaliação predita
0,242.0,23.0,4.0,2.0,1.0
1,197.0,15.0,3.0,3.0,1.0
2,67.0,32.0,2.0,1.0,0.0
3,94.0,27.0,3.0,3.0,0.0
4,138.0,5.0,2.0,3.0,1.0
5,314.0,50.0,4.0,5.0,1.0


Agora, veremos uma animação ilustrando algumas das primeiras iterações do processo de treinamento do perceptron e atualização dos pesos e bias. (A renderização demora alguns minutos, então dá tempo de ir tomar um café.)

In [17]:
%%manim -v WARNING --disable_caching -qm ManimPerceptron

PERCEP_RADIUS = 1.3
RENDER_STEPS = 6

class InputGraph:
    def __init__(self, arrow_num, input_num=0, weight_num=0):
      self.arrow = Arrow(color=BLUE, start=(-6, 3 - (6*arrow_num/(features - 1)), 0), end=ORIGIN, buff=PERCEP_RADIUS)
      self.weight_tracker = ValueTracker(weight_num)
      self.weight_var = DecimalNumber(0, color=BLUE, font_size=28, include_sign=True, group_with_commas=False).next_to(self.arrow.get_midpoint(), UP).add_updater(lambda wei: wei.set_value(self.weight_tracker.get_value()))
      self.input_tracker = ValueTracker(int(input_num))
      self.input_var = DecimalNumber(0, font_size=28, group_with_commas=False).next_to(self.arrow.start).align_to(self.arrow.start, LEFT).add_updater(lambda inp: inp.set_value(self.input_tracker.get_value()))

    def get_graphics(self):
      return (self.arrow, self.input_var, self.weight_var)

    def get_input_tracker(self):
      return self.input_tracker

    def get_weight_tracker(self):
      return self.weight_tracker

    def set_input_tracker(self, inp):
      self.input_tracker.set_value(inp)

    def set_weight_tracker(self, wei):
      self.weight_tracker.set_value(wei)

class ManimPerceptron(Scene):
  def construct(self):
    input_values = data_stream['input'][0]
    weight_values = data_stream['perceptron'][0]['W'].tolist()
    bias_value = data_stream['perceptron'][0]['b']

    circle = Circle(color=WHITE, radius=PERCEP_RADIUS)
    percep_formula = MathTex(r"\sum_{i} w_i \cdot x_{i} + b \geq 0", font_size=25)

    bias_arrow = Arrow(color="#d695ed", start=(0,4,1), end=ORIGIN, buff=PERCEP_RADIUS)
    bias_tracker = ValueTracker(bias_value)
    bias_var = DecimalNumber(0, font_size=28, color="#d695ed", include_sign=True, group_with_commas=False)
    bias_var.add_updater(lambda x: x.set_value(float(bias_tracker.get_value())))
    bias_var.next_to(bias_arrow)

    output_arrow = Arrow(color=ORANGE, start=ORIGIN, end=(5, 0, 0), buff=PERCEP_RADIUS)
    output_tracker = ValueTracker(0)
    output_var = Integer(0, font_size=32, include_sign=False, group_with_commas=False)
    output_var.add_updater(lambda x: x.set_value(float(output_tracker.get_value())))
    output_var.next_to(output_arrow.end, buff=0)

    prediction = DecimalNumber(0, num_decimal_places=2, font_size=36, include_sign=False, group_with_commas=False).arrange(RIGHT, center=True)
    prediction_tracker = ValueTracker(0)
    prediction.add_updater(lambda x: x.set_value(float(prediction_tracker.get_value())))
    
    expected_equal = Integer(0, font_size=32, include_sign=False, group_with_commas=False, color=GREEN).next_to(output_var, DOWN)
    expected_diff = Integer(0, font_size=32, include_sign=False, group_with_commas=False, color=RED).next_to(output_var, DOWN)
    expected_tracker = ValueTracker(0)
    expected_equal.add_updater(lambda x: x.set_value(float(expected_tracker.get_value())))
    expected_diff.add_updater(lambda x: x.set_value(float(expected_tracker.get_value())))

    backprop_arrow = ArcBetweenPoints((4.5, -1.7, 0), (-3, -3, 0), radius=-10, color=RED)
    backprop_arrow.add_tip()
    backprop_w = MathTex(r"w_i = w_i + \eta \cdot(y_{real} - y_{predito}) \cdot x_i", font_size=26, color=RED)
    backprop_b = MathTex(r"b_i = b_i + \eta \cdot(y_{real} - y_{predito})", font_size=26, color=RED)
    backprop_b.next_to(backprop_arrow.get_midpoint(), UP, buff=0.7)
    backprop_w.next_to(backprop_b, UP, 0.2)
    backprop_no_need = Tex(r"Pesos mantidos", font_size=26, color=GREEN).next_to(backprop_arrow.get_midpoint(), UP, buff=0.7)

    self.play(Create(circle), Write(percep_formula))

    input_graph_list = []
    for i in range(features):
      input_graph_list.append(InputGraph(i, input_values[i], weight_values[i]))
      arrow, input_var, weight_var = input_graph_list[i].get_graphics()
      self.play(GrowArrow(arrow), Write(input_var), Write(weight_var), run_time=0.25)
             
    self.play(Write(bias_var), Write(output_var), GrowArrow(bias_arrow), GrowArrow(output_arrow), run_time=0.25)

    self.wait(3)

    self.play(FadeTransform(percep_formula, prediction), run_time=0.5)

    self.wait(2)

    for i in range(1, min(RENDER_STEPS + 1, len(data_stream['perceptron']))):
      for j in range(features):
        arrow, input_var, weight_var = input_graph_list[j].get_graphics()
        self.play(FadeTransform(input_var, prediction),
                  FadeTransform(weight_var, prediction),
                  prediction_tracker.animate.increment_value(input_var.get_value()*weight_var.get_value()),
                  run_time=0.75)
        input_graph_list[j].set_input_tracker(data_stream['input'][i][j])
        input_graph_list[j].set_weight_tracker(data_stream['perceptron'][i]['W'][j])
      self.play(FadeTransform(bias_var, prediction),
                prediction_tracker.animate.increment_value(bias_var.get_value()),
                run_time=0.75)
      self.wait(1)
      bias_tracker.set_value(data_stream['perceptron'][i]['b'])
      now_prediction = predict(data_stream['input'][i-1], data_stream['perceptron'][i-1])
      now_expected = data_stream['output'][i-1]
      self.play(FadeTransform(prediction, output_var),
                output_tracker.animate.set_value(now_prediction))
      expected_tracker.set_value(now_expected)
      self.wait(1)
      if(now_prediction == now_expected):
        self.play(Write(expected_equal), run_time=0.12)
        self.play(Indicate(expected_equal), run_time=0.5)
        self.play(Write(backprop_no_need), run_time=0.5)
      else:
        self.play(Write(expected_diff), run_time=0.12)
        self.play(Indicate(expected_diff), run_time=0.5)
        self.play(Write(backprop_arrow), Write(backprop_b), Write(backprop_w))
      self.wait(1)
      prediction_tracker.set_value(0)
      for j in range(features):
        arrow, input_var, weight_var = input_graph_list[j].get_graphics()
        self.play(Write(input_var), Write(weight_var), run_time=0.12)
      self.play(Write(bias_var), run_time=0.12)
      self.wait(4)
      if(now_prediction == now_expected):
        self.play(FadeOut(expected_equal), FadeOut(backprop_no_need), Write(prediction), run_time=0.5)
      else:
        self.play(FadeOut(expected_diff), FadeOut(backprop_arrow), FadeOut(backprop_b), FadeOut(backprop_w), Write(prediction), run_time=0.5)
      self.wait(2)
    
    self.wait(3)